In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import os

# Charger séquences
npz = np.load("plank_model/data/plank_sequences.npz")
X = torch.tensor(npz["X"], dtype=torch.float32)
y = torch.tensor(npz["y"], dtype=torch.float32)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Modèle LSTM
class PlankLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(X.shape[2], 128, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1]   # last time step
        return torch.sigmoid(self.fc(out))

model = PlankLSTM()

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

epochs = 5
for e in range(epochs):
    for xb, yb in loader:
        pred = model(xb)
        loss = loss_fn(pred.squeeze(), yb)

        opt.zero_grad()
        loss.backward()
        opt.step()

    print(f"Epoch {e+1}/{epochs} - Loss: {loss.item()}")

save_dir = r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\plank_model\plank_model\model"
os.makedirs(save_dir, exist_ok=True)
torch.save(model.state_dict(), os.path.join(save_dir, "plank_lstm.pt"))


Epoch 1/5 - Loss: 0.00023806141689419746
Epoch 2/5 - Loss: 9.843405132414773e-05
Epoch 3/5 - Loss: 5.4432934120995924e-05
Epoch 4/5 - Loss: 3.526929140207358e-05
Epoch 5/5 - Loss: 2.467684134899173e-05
